# Import library

In [12]:
import torch
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import random
import torchvision
from typing import Any
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
from sklearn.metrics import accuracy_score



# Setup device and tensorboard

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
writer = SummaryWriter("logs_classification")
torch.manual_seed(42)

# Customized Dataset

In [14]:
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None) -> None:
        super().__init__()
        self.root_dir = root_dir
        self.transform = transform
        
        self.image_paths =[]
        self.image_paths.extend(glob.glob(os.path.join(root_dir,"with","*.jpg")))
        self.image_paths.extend(glob.glob(os.path.join(root_dir,"without","*.jpg")))
        random.shuffle(self.image_paths)
    def __len__(self) -> int:
        return len(self.image_paths)
    def __getitem__(self, index) -> Any:
        image_path = self.image_paths[index]
        
        label = 1 if image_path.split(os.sep)[-2] == "with" else 0
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
            
        return [image, label]
        

# Train one epoch

In [15]:
def train_model(model, loss_function, optimizer, train_loader, epoch=1):
    model.train()
    running_loss = 0.0
    total_samples = 0
    y_true = []
    y_pred = []
    for data in train_loader:     
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # Clear gradient
        optimizer.zero_grad()
        # Calculate logits
        outputs = model(images)
        # Calculate loss
        loss = loss_function(outputs, labels)
        # Calculate gradient from loss
        loss.backward()
        # Update weight
        optimizer.step()
        

        # Calculate loss
        running_loss += loss.item() * images.size(0)
        # Calculate total sample in data_loader
        total_samples += images.size(0)
        # Calculte y_predict for evaluation
        predicted = torch.argmax(outputs, dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.detach().cpu().numpy())
    writer.add_scalar("train/loss", running_loss/total_samples, epoch)
    writer.add_scalar("train/accuracy", accuracy_score(y_true, y_pred), epoch)
    return running_loss/total_samples

# Eval model



In [16]:
def eval_model(model, loss_function, test_dataloader, epoch):
    model.eval()
    total_samples = 0
    total_loss = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)

            # Calculate logits
            outputs = model(images)
            
            # Calculate loss of outputs and y_true
            loss = loss_function(outputs, labels)
            total_loss += loss.item() * images.size(0)
            # Calculate total sample
            total_samples += images.size(0)
            
            # Calculte y_predict for evaluation
            predicted = torch.argmax(outputs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.detach().cpu().numpy())
    writer.add_scalar("test/loss", total_loss/total_samples, epoch)
    writer.add_scalar("test/accuracy", accuracy_score(y_true, y_pred), epoch)
    return total_loss/total_samples

# Model

In [17]:
model = torchvision.models.resnet34(num_classes=2)
model = model.to(device)
print(device)

cuda:0


# Optimizer


In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Loss function

In [19]:
loss_function = torch.nn.CrossEntropyLoss()

# Train

In [20]:
root_dir = os.path.join(os.getcwd(), "dataset", "gan_makeup_data_96")
transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop([96,96]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
        torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])
epoch = 20
dataset = ImageDataset(root_dir, transform)

# Perform train-test split
train_dataset, test_dataset = random_split(dataset, [0.9,0.1])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

for i in range(20):
    train_loss = train_model(model, loss_function, optimizer, train_loader, i)
    test_loss = eval_model(model, loss_function, test_loader, i)
    print(f"epoch {i}: train_loss {train_loss}, test_loss {test_loss}")
writer.flush()
writer.close()



epoch 0: train_loss 0.8193305544802146, test_loss 1.022890947930824
epoch 1: train_loss 0.6989265765115231, test_loss 0.6897348940097792
epoch 2: train_loss 0.677312676819048, test_loss 0.7207033779763956
epoch 3: train_loss 0.6479702967882521, test_loss 0.7563864283847369
epoch 4: train_loss 0.6482601230512889, test_loss 0.7302073086461713
epoch 5: train_loss 0.6382098374228159, test_loss 0.7043684086491985
epoch 6: train_loss 0.6357455623204583, test_loss 0.7071269874199195
epoch 7: train_loss 0.6114231245910434, test_loss 0.7711073283775611
epoch 8: train_loss 0.6204496950047895, test_loss 0.6886007596270829
epoch 9: train_loss 0.6121203696344776, test_loss 0.7804821575841596
epoch 10: train_loss 0.6175203530962281, test_loss 0.6901030241069706
epoch 11: train_loss 0.6001474616905671, test_loss 0.6849511870041421
epoch 12: train_loss 0.5926800183894864, test_loss 0.6612219327056463
epoch 13: train_loss 0.5861191100335011, test_loss 0.702456780018345
epoch 14: train_loss 0.5956047366

In [ ]:
# print(len(dataset))

# fig, axes = plt.subplots(10, 10, figsize=(10,10))
# for i in range(10):
#     for j in range(10):
#         image = torchvision.transforms.ToPILImage()(dataset[i*10+j][0])
#         label = dataset[i*10+j][1]
#         axes[i][j].imshow(image)
#         axes[i][j].axis("off")
#         axes[i][j].set_title(label)
# plt.show()



In [21]:
# print(len(dataset))

# fig, axes = plt.subplots(10, 10, figsize=(10,10))
# for i in range(10):
#     for j in range(10):
#         image = torchvision.transforms.ToPILImage()(dataset[i*10+j][0])
#         label = dataset[i*10+j][1]
#         axes[i][j].imshow(image)
#         axes[i][j].axis("off")
#         axes[i][j].set_title(label)
# plt.show()



In [22]:
import numpy as np

def cross_entropy_loss(logits, targets):
    num_samples = targets.shape[0]

    softmax_probs = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
    a = np.arange(num_samples)
    b = softmax_probs[np.arange(num_samples), targets]
    log_probs = -np.log(softmax_probs[np.arange(num_samples), targets])

    loss = np.sum(log_probs) / num_samples
    return loss
logits = np.array([[2,4],
                  [5,2]], dtype=np.float32)
target = np.array([0,1])

loss = cross_entropy_loss(logits, target)
print(loss)

logits = torch.tensor([[2,4],
                  [5,2]], dtype=torch.float32)
target = torch.tensor([0,1])

softmax = torch.nn.Softmax(dim=1)
softmax_logits = softmax(logits)
print(softmax_logits)
loss_function = torch.nn.CrossEntropyLoss()
loss = loss_function(logits, target)
print(loss)

2.5877575874328613
tensor([[0.1192, 0.8808],
        [0.9526, 0.0474]])
tensor(2.5878)
